In [25]:
# All imports
import carla
import time
import cv2
import numpy as np

import sys
sys.path.append('C:\\Users\\eglm\\Documents\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [26]:
# Connect to the carla

client = carla.Client('localhost', 2000)

In [27]:
world = client.get_world()
spanw_points = world.get_map().get_spawn_points()

# look for a blueprint of mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# spanw a car
start_point = spanw_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0],start_point)

# setup camera

CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans, attach_to= vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [28]:
point_a = start_point.location

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

# get de longest possible route
distance = 0
for loc in spanw_points:

    cur_route = grp.trace_route(point_a,loc.location)
    if len(cur_route)>distance:
        distance = len(cur_route)
        route = cur_route

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color = carla.Color(r=0,g=255,b=0), life_time = 60.0,
        persistent_lines=True)

In [29]:
for way_point in route:
    vehicle.set_transform(way_point[0].transform)
    cv2.imshow('fake self-driving', camera_data['image'])
    cv2.waitKey(1)

time.sleep(2)
cv2.destroyAllWindows()
camera.stop()

# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()